# Продвинутый pandas
## Задание 1

In [67]:
import pandas as pd

df = pd.read_csv('visit_log.csv', sep=';')

def source_type(row):
    if row['traffic_source'] in ('yandex', 'google'):
        return 'organic'
    
    if row['traffic_source'] in ('paid', 'email'):
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'
    
    return row['traffic_source']

df['source_type'] = df.apply(source_type, axis=1)
df.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


## Задание 2

In [2]:
import pandas as pd

df = pd.read_csv('URLs.txt', delimiter = "\t")

# На всякий случай экранировал дефис тоже
df[df.url.str.contains(r'\/\d{8}\-', regex=True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3
### Решение №1 - Простое (сначала вычисляем все в одном месте, затем фильтруем)

In [64]:
%%time
ratings = pd.read_csv('ml-latest-small/ratings.csv')

user_ratings = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = ['max', 'min', 'count'], fill_value = 0)
filtered_user_ratings = user_ratings[user_ratings['count']['timestamp'] > 100]

life_time = filtered_user_ratings['max']['timestamp'].mean() - filtered_user_ratings['min']['timestamp'].mean()

print(f'Среднее время жизни пользователей:\nВ секундах - {round(life_time)}\nВ часах - {round(life_time/3600)}\nВ днях - {round(life_time/3600/24)}')

Среднее время жизни пользователей:
В секундах - 40080507
В часах - 11133
В днях - 464
CPU times: user 57.5 ms, sys: 3.88 ms, total: 61.4 ms
Wall time: 67.2 ms


### Решение №2 - Оптимальное (сначала фильтруем, затем вычисляем min/max)?

In [65]:
%%time
user_ratings = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = 'count', fill_value = 0)

# Фильтруем
id_users_gt_100_ratings = user_ratings[user_ratings['timestamp'] > 100]
users_gt_100_ratings = ratings[ratings.userId.isin(id_users_gt_100_ratings.index)]

# Вычисляем
max_min_users_gt_100_ratings = users_gt_100_ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = ['max', 'min'], fill_value = 0)
life_time = max_min_users_gt_100_ratings['max']['timestamp'].mean() - max_min_users_gt_100_ratings['min']['timestamp'].mean()

print(f'Среднее время жизни пользователей:\nВ секундах - {round(life_time)}\nВ часах - {round(life_time/3600)}\nВ днях - {round(life_time/3600/24)}')

Среднее время жизни пользователей:
В секундах - 40080507
В часах - 11133
В днях - 464
CPU times: user 31 ms, sys: 91 µs, total: 31.1 ms
Wall time: 32.3 ms


## Задание 4

In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

print('Без адреса =================')
joined1 = client_base.loc[:,['client_id']].merge(air, on='client_id', how='left').\
merge(auto, on='client_id', how='left').\
merge(rzd, on='client_id', how='left').fillna(0)
print(joined1)
print('С адресом =================')
joined2 = client_base.merge(air, on='client_id', how='left').\
merge(auto, on='client_id', how='left').\
merge(rzd, on='client_id', how='left').fillna(0)
print(joined2)

Без адреса =================
   client_id  air_revenue  auto_revenue  rzd_revenue
0        111          0.0           0.0       1093.0
1        112          0.0           0.0       2810.0
2        113          0.0       57483.0      10283.0
3        114          0.0          83.0       5774.0
4        115         81.0         912.0        981.0
5        116          4.0        4834.0          0.0
6        117         13.0          98.0          0.0
7        118        173.0           0.0          0.0
С адресом =================
   client_id          address  air_revenue  auto_revenue  rzd_revenue
0        111  Комсомольская 4          0.0           0.0       1093.0
1        112   Энтузиастов 8а          0.0           0.0       2810.0
2        113  Левобережная 1а          0.0       57483.0      10283.0
3        114          Мира 14          0.0          83.0       5774.0
4        115        ЗЖБИиДК 1         81.0         912.0        981.0
5        116    Строителей 18          4.0    